In [1]:
import nltk
import collections
import numpy as np
from keras.preprocessing import sequence
from sklearn.model_selection import train_test_split
word_freqs = collections.Counter()

/Users/championlin/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
from keras.layers.core import Activation, Dense
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM
from keras.models import Sequential

In [9]:
max_len = 0
nb_sample = 0

In [10]:
# use to clean sentence
def CleanLines(line):  
    identify1 = str.maketrans(',', ' ')  
    identify2 = str.maketrans(':', ' ')  
    identify3 = str.maketrans('.', ' ')
    identify4 = str.maketrans('!', ' ')  
    identify5 = str.maketrans('?', ' ')  
#     delEStr = string.punctuation   
    cleanLine = line.translate(identify1) 
    cleanLine = cleanLine.translate(identify1) 
    cleanLine = cleanLine.translate(identify2) 
    cleanLine = cleanLine.translate(identify3) 
    cleanLine = cleanLine.translate(identify4)
    cleanLine = cleanLine.translate(identify5)
#     cleanLine =cleanLine.translate(delEStr) 
    return cleanLine  

In [11]:
# count words
label = []
sentence = []
with open('./data/train.txt', 'r') as f:
    for line in f:
        line = line.lower()
        spl = line.strip().split('/t')
        label.append(spl[0][0]) 
        sentence.append(spl[0][1:])
        line = CleanLines(line)
        words = nltk.word_tokenize(line)
        if len(words) > max_len:
            max_len = len(words)
        for word in words:
            word_freqs[word] += 1
        nb_sample += 1
            
print('max sentence len:', max_len)
print('number of words:', len(word_freqs))
print('number of samples:', nb_sample)

max sentence len: 242
number of words: 88782
number of samples: 50277


In [7]:
#set params
MAX_SENTENCE = 242
MAX_WORD = 85000
vocab_size = MAX_WORD + 2

In [8]:
# make dictionary
word2index = {x[0]: i+2 for i, x in enumerate(word_freqs.most_common(MAX_WORD))}
word2index["PAD"] = 0
word2index["UNK"] = 1
index2word = {v:k for k, v in word2index.items()}

In [16]:
# word to index
X = np.empty(nb_sample, dtype=list)
y = np.zeros(nb_sample)
i=0
with open('./data/train.txt', 'r') as f:
    for line in f:
        line = line.lower()
        line = CleanLines(line)
        words = nltk.word_tokenize(line)
        seqs = []
        for word in words:
            if word in word2index:
                seqs.append(word2index[word])
            else:
                seqs.append(word2index["UNK"])
        X[i] = seqs
        i += 1

In [17]:
#index to sequence
X = sequence.pad_sequences(X, maxlen=MAX_SENTENCE)

In [18]:
label = np.array(label)

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, label, test_size=0.2, random_state=42)

In [ ]:
a_train, a_valid, b_train, b_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [ ]:
embedding_size = 128
layers = 64

model = Sequential()
model.add(Embedding(vocab_size, embedding_size, input_length=MAX_SENTENCE))
model.add(LSTM(layers, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1))
model.add(Activation("sigmoid"))
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
epoches = 10
batches = 320
model.fit(a_train, b_train, batch_size=batches, epochs=epoches, validation_data=(a_valid, b_valid))

In [24]:
model.evaluate(X_test, y_test)

10056/10056 [==============================] - 27s 3ms/step


[2.899936153192604, 0.5096459824980112]

In [23]:
# from keras.models import load_model
model = load_model('/Users/championlin/Downloads/new_weights (1).h5')

/Users/championlin/anaconda3/lib/python3.6/site-packages/keras/models.py:291: UserWarning: Error in loading the saved optimizer state. As a result, your model is starting with a freshly initialized optimizer.
  warnings.warn('Error in loading the saved optimizer '


In [14]:
def input_sentence():
    while True:
        sent = input('Enter the sentence:')
        X = np.empty(1,dtype=list)
        y = np.zeros(1)
        i=0
        sent = sent.lower()
        words = nltk.word_tokenize(sent)
        seqs = []
        for word in words:
            if word in word2index:
                seqs.append(word2index[word])
            else:
                seqs.append(word2index["UNK"])
        X[i] = seqs
        i += 1
        X = sequence.pad_sequences(X, maxlen=MAX_SENTENCE)

        labels = [int(np.round(x[0])) for x in model.predict(X) ]
        label2word = {1:'positive', 0:'negative'}
        print('{}'.format(label2word[labels[0]]))
        print(model.predict(X))

In [15]:
input_sentence()

Enter the sentence:love
negative
[[0.31617326]]
Enter the sentence:like
negative
[[0.31617326]]
Enter the sentence:dadads
negative
[[0.31617326]]
Enter the sentence:addd
negative
[[0.31617326]]


KeyboardInterrupt: 